# ランダム変数と全要素生産性

<div name="html-admonition" style="font-size: 0.8em">
<input type="button" onclick="location.href='https://translate.google.com/translate?hl=&sl=ja&tl=en&u='+window.location;" value="Google translation" style="color:#ffffff;background-color:#008080; height:25px" onmouseover="this.style.background='#99ccff'" onmouseout="this.style.background='#008080'"/> in English or the language of your choice.
</div><br>

In [ ]:
import japanize_matplotlib
import numpy as np
import pandas as pd
import py4macro
import statsmodels.formula.api as smf

# numpy v1の表示を使用
np.set_printoptions(legacy='1.21')
# 警告メッセージを非表示
import warnings
warnings.filterwarnings("ignore")

## はじめに

この章の目的は２つある。
第一に，マクロ経済分析におけるランダム変数の役割と変数の持続性について考える。
第二に，景気循環を理解する上で全要素生産性の役割が強調される研究があるが，その導入として全要素生産性の特徴について検討する。またソロー・モデルを使い，全要素生産性が確率的に変動する簡単な景気循環モデルのシミュレーションを通してマクロ変数の特徴を考察する。

## ランダム変数の役割

In [ ]:
# データの読み込み
df = py4macro.data('jpn-q')

# トレンドからの％乖離のデータの作成
df['gdp_cycle'] = np.log( df['gdp'] ) - py4macro.trend( np.log(df['gdp']) )

# プロット
ax = ( 100 * df['gdp_cycle'] ).plot(title='GDPのトレンドからの乖離（％）')
ax.axhline(0, c='red')
pass

このようなGDPの変動を説明する方法として２つある。
1. 決定的（deterministic）な過程で生成された時系列として捉える。
1. ランダム（stochastic）な過程で生成された時系列として捉える。

１の考えによると，差分方程式$x_{t+1}=f(x_t)$のように来期のGDPは今期のGDPに依存している。マクロ変数の持続性を捉えるには必要不可欠な特徴と言えるだろう。しかし，今期のGDP（$x_t$）が与えられると来期のGDP（$x_{t+1}$）は自ずと決定さることになり，来期のGDPの予測は簡単なものとなる。もしそうであれば政策運営は非常に簡単であろうが，現実はそうではない。そういう意味では，１は現実を十分に捉えることができていない。

一方，２の考えはランダム変数の実現値の連続としてGDPが観測され，今期のGDPが与えられても来期のGDPにはランダムな要素があるため，予測が難しいという特徴がある。この特徴こそが，マクロ変数の性質を捉えるには必要な要素であり，上のプロットに現れていると考えられる。主流のマクロ経済学では，消費者や企業の行動はフォーワード・ルッキングであり（将来を見据えた最適化行動であり），且つ経済全体ではランダムな（不確実な）要素が重要な役割を果たしているという考え基づいている。そのような分析枠組みの中で，景気循環のメカニズムを解明することが目的となっている。

以下では，まずランダム変数生成過程を考え，短期分析における全要素生産性の役割について説明する。またSolowモデルを使い簡単な景気循環モデルを展開する。

## ホワイト・ノイズ

### 説明

時系列のランダム変数$\varepsilon_t$，$t=0,1,2,3,\cdots$を考えよう。例えば，レストランの経営者の収入。ビジネスにはリスク（競争相手の出現やコロナ感染症問題）があるため変動すると考えるのが自然である。$\varepsilon_t$は$t$毎にある分布から抽出されると考えることができる。次の３つの性質を満たしたランダム変数をホワイト・ノイズ（White Noise）と呼ぶ。
1. 平均は`0`：$\quad\text{E}\left[\varepsilon_t\right]=0$
1. 分散は一定：$\text{ E}\left[\varepsilon_t^2\right]=\sigma^2$（$\sigma^2$に$t$の添字はない）
1. 自己共分散（自己相関）は`0`：$\text{ E}\left[\varepsilon_t \varepsilon_{t-s}\right]=0$（全ての$s\ne 0$に対して）。

＜コメント＞
* 平均`0`と分散が$\sigma^2$のホワイト・ノイズを次のように表記する。

    $$
    \varepsilon_t\sim\textit{WN}(0,\sigma^2)
    $$
    
* ホワイト・ノイズの例：
    * 平均`0`，分散$\sigma^2$の正規分布から抽出された$\varepsilon_t$
    * 最小値`10`，最大値`100`からからランダム抽出された$\varepsilon_t$
* 正規分布や一様分布ではない分布でもホワイト・ノイズになる。
* ホワイト・ノイズは独立同分布（Indepent and Identically Distributed）の１つである。

### 平均と分散

以下では実際にランダム変数を発生させ，そのプロットと統計的な特徴について考察する。まず，平均と分散について確認するために，その準備として次のコードを実行しよう。

In [ ]:
rng = np.random.default_rng()

`rng`はランダム変数を生成する「種」となるオブジェクトと理解しよう。

```{note}
`NumPy`を使いランダム変数を生成する方法については，[「経済学のためのPython入門」のNumpy: ランダム変数](https://py4basics.github.io/8_NumPy_random.html)を参考にしてください。
```

標準正規分布からのランダム変数を`n`個抽出しプロットしてみよう。

In [ ]:
n = 100

vals = rng.normal(size=n)

dfwn = pd.DataFrame({'WN':vals})

dfwn.plot(marker='.')
pass

２つの特徴がある。
* 平均`0`：`0`を中心に周辺を動いている。
* 分散一定：`0`から外れても`0`に戻っている。標準正規分布の場合、`-2`から`2`の間の値を取る確率が約`95`％になる性質からも理解できるだろう。

平均と分散を計算してみよう。

In [ ]:
dfwn['WN'].mean(), dfwn['WN'].var()

標準正規分布の母集団からのサンプル統計量であるため誤差が発生していることがわかる。

２つの特徴をを確認するために`Pandas`の`plot()`を使ってヒストグラムを描いてみよう。

In [ ]:
dfwn.plot(kind='hist', bins=30)
pass

概ね`0`を中心に左右対象に動いている。即ち，平均`0`を反映している。また，`0`から離れている観測値が少ないことが分かる。これは`0`方向に戻ることを示している。これは分散が一定であるためである。この性質は`n`を`500`や`1000`などの大きな数字に設定するより一層分かりやすいだろう。

### 自己共分散（自己相関）

#### 復習

次に自己共分散について考えるために，２つのランダム変数`X_i`と`Y_i`、$i=1,2,3,\cdots,n$を使い分散と共分散の関係を整理しよう。各変数の分散と共分散を次のように表記しよう。

$$
X_i\text{の分散}=\sigma_X^2,
\qquad
Y_i\text{の分散}=\sigma_Y^2
\qquad
X_i\text{と}Y_i\text{の共分散}=\sigma_{XY}
$$

ここで，次の定義は覚えているだろう。

$$
X_i\text{の標準偏差}=\sigma_X,
\qquad
Y_i\text{の標準偏差}=\sigma_Y
$$

また，次も成立する。

$$
\begin{alignat}
X_i\text{と}X_i\text{の共分散}&=\sigma{XX}=\sigma_X^2=X_i\text{の分散}\\
Y_i\text{と}Y_i\text{の共分散}&=\sigma{YY}=\sigma_Y^2=Y_i\text{の分散}
\end{alignat}
$$

共分散は２変数の相関度を示す指標だが，同じ変数同士の場合は分散となる。

前章でも説明したが，相関度を表す**相関係数**は次のように定義される。

$$
X_i\text{と}Y_i\text{の共分散}=
\rho_{XY}=
\frac{
    \sigma_{XY}
    }{
    \sigma_X
    \sigma_Y
    }
$$

次の特徴がある。
* 分母は必ず正となるので，分子の共分散が符号を決定する。
* $\rho_{XY}$の値は`-1`から`1`の間の値を取る。

さあ、ここでホワイト・ノイズのランダム変数を$\varepsilon_t$として、次のように置いてみよう。
* $X_i=\varepsilon_t,\quad t=0,1,2,3,\cdots$
* $Y_i=\varepsilon_{t-s},\quad s=1,2,3,\cdots$

$X_i$と$Y_i$のそれぞれの分散は，`s`期ずれているだけの同じランダム変数$\varepsilon$である。
更に，$X_i$と$Y_i$を使うと，自己共分散は次のように定義できる。

$$
X_i\text{と}Y_i\text{の共分散}=\sigma_{XY}
=\sigma_{\varepsilon_t,\varepsilon_{t-s}}
=s\text{期間離れた}\varepsilon\text{の自己共分散}
$$

即ち，自己共分散はランダム変数とその変数の`s`期前の値との**自己相関**の度合を表た指標ということになる。次に，$\varepsilon_t$の相関係数は

$$
\rho_{\varepsilon}(s)\equiv\rho_{\varepsilon_t,\varepsilon_{t-s}}
=\frac{
    \sigma_{\varepsilon_t,\varepsilon_{t-s}}
    }{
    \sigma_{\varepsilon_t}
    \sigma_{\varepsilon_{t-s}}
    }
$$

で与えられる。自己相関係数と呼ばれる場合もあるが，より一般的には**自己相関関数**と呼ばれる。「関数」と呼ばれる理由は，$\sigma_{\varepsilon_t,\varepsilon_{t-s}}$は`s`の関数として考えることができるためだ。呼称よりも重要なのが数値の解釈である。
* $\rho_{\varepsilon}(s)=0,\;s=1,2,3,\cdots$：何期離れたとしても自己相関はなしという意味であり，これがまさしくホワイト・ノイズの３つ目の性質である。
* $\rho_{\varepsilon}(s)>0,\;s=1,2,3,\cdots$：今期と`s`期前の値は正の相関があるということを示す。解釈しやすくするために`s`$=1$の場合を考えると，$\varepsilon_{t-1}$の値が大きければ（小さければ），$\varepsilon_{t}$も大きい（小さい）傾向にあるという意味であり，`s`期前の影響が強ければ，自己相関係数の絶対値は大きくなる。経済学では，この性質を**持続性**（persisitence）と呼ぶ。持続性は多くのマクロ変数の重要な特徴となっている。
* $\rho_{\varepsilon}(s)<0,\;s=1,2,3,\cdots$：今期と`s`期前の値は負の相関があるということを示す。`s`$=1$の場合を考えると，$\varepsilon_{t-1}$の値が大きければ（小さければ），$\varepsilon_{t}$は小さい（大きい）傾向にあるという意味であり，`s`期前の影響が強ければ，自己相関係数の絶対値は大きくなる。

#### プロットと計算

では，`1`期違いの自己共分散$\sigma_{\varepsilon_t \varepsilon_{t-1}}$を計算するために，`dfwn`の列`WN`を`1`期シフトさせた新たな列`WNlag`を作成しよう。

In [ ]:
dfwn['WNlag'] = dfwn['WN'].shift()
dfwn.head()

`WNlag`の値は`WN`の値が`1`期シフトしていることが分かる。
`0`番目の行の`WN`が初期値であり、同じ値が`1`番目の行の`WNlag`に入っている。
即ち、`1`番目の行を見ると`WN`には`1`期の値、、`WNlag`には初期（`0`期）の値がある。
同様に、各行の`WN`には`t`期の値、`WN`に`t-1`期の値が入っている。

この２つの列を使って散布図を描いてみる。`Pandas`の`plot()`で横軸・縦軸を指定し，引数`kind`で散布図を指定するだけである。

In [ ]:
dfwn.plot(x='WNlag', y='WN', kind='scatter')
pass

自己共分散がゼロであれば，ランダムに散らばっているはずであり，何らかのパターンも確認できないはずである。

ここで異なるプロットの方法を紹介する。`Pandas`のサブパッケージ`plotting`にある`lag_plot()`関数を使うと共分散の強さを示す図を簡単に作成することが可能となる。
* 必須の引数：１つの列
* オプションの引数：`lag`は時間ラグ（デフォルトは`1`）

In [ ]:
pd.plotting.lag_plot(dfwn['WN'])
pass

* 横軸：`t`期の値
* 縦軸：`t+1`期の値

２つの図は同じであることが確認できる。

分散・自己共分散を計算してみよう。

In [ ]:
dfwn.cov()

左上と右下の対角線上にあるのは、`WN`と`WNlag`の不偏分散であり、`1`に近い値となっている。両変数は標準正規分布（分散`1`）から生成したためであり、`1`にならないのはサンプルの誤差である。（`WN`と`WNlag`の不偏分散は異なるが、その理由は？）一方、右上と左下の値（同じ値となる）が**自己共分散**であり、非常に小さな値であるが、自己共分散は`0`という仮定を反映している。

`dfwn.cov()`を使って自己相関係数を計算することができるが、一点注意する必要がある。`dfwn`の`0`行目には欠損値があるため、自己共分散を計算する際にその行は使われていない。同様に、自己相関係数を計算する際は欠損値がある行を使わずに計算する必要がある。この点に注意して、次のコードで計算することができる。

In [ ]:
varcov = dfwn.dropna().cov()
varcov.iloc[0,1] / ( varcov.iloc[0,0]**0.5 * varcov.iloc[1,1]**0.5 )

メソッド`.corr()`を使っても同じ結果を得ることができる。

In [ ]:
dfwn.corr()

また、`dfwon['WN']`のメソッド`autocorr()`を使い、直接**自己相関係数**を計算することも可能である。

In [ ]:
dfwn['WN'].autocorr()

## マクロ変数の持続性

GDPや構成要素を含め、マクロ変数の重要な特徴が持続性である。
上で説明したように、持続性は自己相関と同義であり、自己相関関数で測ることができる。
ここでは、主なマクロ変数の持続性を計算し、その度合いを確認する。

### GDPと構成要素

まずGDPと構成要素のトレンドからの乖離率の持続性を考察するが、次のコードは乖離率の変数を作成する。

In [ ]:
var_lst1 = ['gdp',
            'consumption',
            'investment',
            'government',
            'exports',
            'imports']

df = py4macro.data('jpn-q')

for v in var_lst1:
    
    df[v+'_log'] = np.log( df.loc[:,v] )
    df[v+'_log_trend'] = py4macro.trend( df.loc[:,v+'_log'] )
    df[v+'_cycle'] = 100 * ( df[v+'_log'] - df[v+'_log_trend'] ) 

前章で解説したコードと同じなので解説は不要だろう。まず視覚的に確認してみよう。

In [ ]:
cycle_lst1 = [s+'_cycle' for s in var_lst1]
df.plot(y=cycle_lst, subplots=True, figsize=(8,10))
pass

上でプロットしたホワイト・ノイズとは印象が異なることが直ぐに分かるのではないだろうか。
`for`ループを使い、自己相関係数を計算してみよう。

In [ ]:
for v in cycle_lst1:
    
    autocorr = df[v].autocorr()         #1
    print(f'{v:<19}{autocorr:>5.3f}')   #2

```{admonision} コードの説明
* `#1`：`.autocorr()`は自己相関係数を計算するメソッド。
* `#2`：f-stringを使っている。
    * `<19`は`v`の文字列の長さを空白を足して`19`にし左詰めにする。
    * `>5`は`autocorr`の文字列の長さに空白を足して5にし右詰めにする。
    * `.2f`は小数点第二位までの表示を設定している。
    * `>5`と`.2f`の順番を逆にするとエラーが発生する。
```

消費を除いて全て0.5以上であり，持続性が確認できる。
特に、投資と輸入の持続性は高いことが分かる。

### インフレ率と失業率

次に、失業率とインフレ率の持続性を確認する。

各変数の乖離を計算するが、注意する点がある。GDPなどの変数と違い、インフレ率と失業率は長期的なトレンドはない。従って、式[](eq:10-decompose_plus)を使い、対数化せずに計算する必要がる。

In [ ]:
var_lst2 = ['inflation', 'unemployment_rate']

for v in var_lst2:

    df[v+'_trend'] = py4macro.trend( df[v] )
    df[v+'_cycle'] = df[v] - df[v+'_trend']

まずプロットしてみよう。

In [ ]:
cycle_lst2 = [v+'_cycle' for v in var_lst2]
df.plot(y=cycle_lst2, subplots=True)

やはり持続性が高いように見える。

In [ ]:
for v in cycle_lst2:
    
    autocorr = df[v].autocorr()
    print(f'{v:<25}{autocorr:.3f}')

数字でも強い持続性が確認できる。

### まとめ

これらの結果からわかることは，ホワイト・ノイズだけではマクロ変数の持続性を説明できないということであり、景気循環を理解するためには自己共分散が正の値になる経済モデルが求められる。では、どのようなモデルなのだろうか？この問いに答えるために、まずは統計的なモデルである自己回帰モデルを考察し、それを出発点として議論を進めていく事にする。

## 自己回帰モデル：AR(1)

#### 説明

ここでは持続性を捉える確率過程である自己回帰モデルを考える。英語で**A**ugo**r**egressive Modelと呼ばれ，AR(1)と表記され，次式で表される。（AR(1)の`(1)`は右辺と左辺の変数は１期しか違わないことを表している。）

$$
y_t=\rho y_{t-1}+\varepsilon_t
$$ (eq:13-ar1)

ここで
* $y_t$：$t$期の$y$の値
* $-1<\rho<1$
* $\varepsilon_t\sim WN(0,\sigma^2)$（ホワイト・ノイズ）

次の特徴がある。第一に、$\varepsilon_t$を所与とすると$y_t$の差分方程式となっている。$-1<\rho<1$となっているため，安定的な過程であることがわかる。即ち，$\varepsilon_t=0$であれば、$y_t$は定常状態である$0$に近づいて行くことになる。しかしホワイト・ノイズである$\varepsilon_t$により、毎期確率的なショックが発生し$y_t$を定常状態から乖離させることになる。第二に、今期の値$y_t$は前期の値$y_{t-1}$に依存しており，その依存度はパラメータ$\rho$によって決定される。この$\rho$こそが$y_t$の持続性の強さを決定することになり，自己回帰モデル[](eq:13-ar1)の自己相関係数は$\rho$と等しくなる。即ち、持続性を捉えるためには、$\rho\ne 0$が必須である。

#### ３つの例：持続性の違い

直感的にマクロ変数の特徴である持続性とは，前期の値が今期の値にどれだけ影響を持っているかを示す。$\rho$の値を変えて持続性の違いを視覚的に確認するための関数を作成しよう。

In [ ]:
def ar1_model(rho, T=100):
    """引数：
            rho: AR(1)の持続性を捉えるパラメータ
            T:   シミュレーションの回数（デフォルト：0）
       戻り値：
            matplotlibの図を示す
            自己相関係数の値を表示する"""
    
    y0 = 0                          # 1
    y_list = [y0]                   # 2
    y = y0                          # 3

    for t in range(1,T):            # 4
        e = rng.normal()            # 5
        y = rho*y + e               # 6
        y_list.append(y)            # 7

    df_ar1 = pd.Series(y_list)      # 8

    ac = df_ar1.autocorr()          # 9
    
    ax_ = df_ar1.plot(marker='.')   # 10
    ax_.set_title(fr'$\rho$={rho}　　自己相関係数：{ac:.3f}',size=20) # 11
    ax_.axhline(0, c='red')         # 12

```{admonition} コードの説明
:class: dropdown

1. `y`の初期値を設定。
2. `y`の値を格納するリスト。初期値を入れてある。
3. 左辺の`y`は`for`ループで使うアップデート用の`y`であり，初期値を割り当てている。
4. `T`期間の`for`ループの開始。変数`t`は(4)と(5)に入っていないので単にループの回数を数えている。
    * `range(1,T)`の`1`はループの計算が$t=1$期の`y`の計算から始まるため。$t=0$期の`y`は(1)で与えられている。
5. `t`期のホワイト・ノイズの生成。
6. 右辺の`e`は`t`期のホワイトノイズであり，`y`前期の`y`となっている。
7. `y_list`に`y`を追加。
8. `y_list`を使って`Series`を作成し`df_ar1`に割り当てる。
9. `.autocorr()`を使い己相関係数を計算し`ac`に割り当てる。
10. `y`のプロット
    * `marker='.'`はデータのマーカーを点に設定。
    * 図の「軸」を`ax_`に割り当てる。
11. 図のタイトルの設定
    * `fr`の`f`は`f-string`を表し，`{rho}`と`{ac:.3f}`に数値を代入する。`:.3f`は小数点第三位までの表示を設定
    * `fr`の`r`は`$\rho$`をギリシャ文字に変換するためののも。`$\rho$`は[LaTeX](https://ja.wikipedia.org/wiki/LaTeX)のコード。
12. `.axhline()`は横線をひくメソッド。
```

In [ ]:
ar1_model(0.1)

この例では$\rho$の値が低いため，$y$は定常状態の0に直ぐに戻ろうとする力が強い。従って、前期の値の今期の値に対する影響力が小さいため、持続性が低い。

In [ ]:
ar1_model(0.5)

$\rho$の値が高くなると、定常状態の$0$に戻ろうとする作用が弱くなり、持続性が強くなることがわかる。

In [ ]:
ar1_model(0.9)

$\rho$が非常に高いため、持続性も非常に強くなっている。即ち、今期の値は前期の値に対する依存度が大きい。

## 全要素生産性

### 説明

マクロ変数の特徴を捉えるにはホワイト・ノイズよりもAR(1)の方がより適しているということが分かった。では，どの様な形でAR(1)をマクロ・モデルに導入すれば良いのだろうか。現在盛んに行われている景気循環研究はDSGEモデル（**D**ynamic **S**tochastic **G**eneral **E**quilibrium Model）と呼ばれるモデルに基づいており、DSGEは名前が示すように次のような特徴がある。
* 時間軸に沿ってマクロ変数の変化を考察する動学的な特徴
* 確率的な要因の重要な役割
* 合理的期待に基づく将来を見据えた消費者と企業の最適行動
* 一般均衡のモデル

そして，DSGE研究の先駆けとなったモデルが実物的景気循環モデル（**R**eal **B**usiness **C**ycles Model; RBCモデル）と呼ばれる理論である。詳細については後の章に譲るが，基本的なアイデアは簡単で全要素生産性（TFP）がAR(1)に従って確率的に変動し，それが消費者と企業の経済行動に影響することにより景気循環が発生するという理論である。ここではデータを使い全要素生産性の性質を考察する。RBCモデルで需要な役割を果たす労働時間についてもデータを使い特徴を探ることにする。

### データ

次の生産関数を仮定する。

$$
Y_t=A_tK_t^aH_t^{1-a}
$$

* $Y_t$：GDP
* $K_t$：資本ストック
* $H_t$：総労働時間（労働者数$\times$労働者平均労働時間）
* $A_t$：TFP

TFPについて解くと次式を得る。

$$
A_t=\dfrac{Y_t}{K_t^aH_t^{1-a}}
$$

景気循環を考えているため、TFPを計算する際には次の点を考慮することが望ましい。
1. 年次データではなく四半期データを使う（年次データでは短期的な変動を捉えることができない）。
1. $H_t$は労働者数ではなく，総労働時間とする（短期的な労働供給の変化を捉えるため）。
1. $K_t$は資本ストックの稼働率を考慮したデータとする（短期的に実際に生産に使われた資本ストックの変化を捉えるため）。

ここでは１と２のみを考慮したデータを使いTFPの変動を考えることにする。これにより資本ストックの稼働率はTFPの一部となってしまうが，RBCモデルの基本的なアイデアは変わらない。

上で読み込んだデータ`df`には次の変数が含まれている（詳細は`GDP：水準・トレンド・変動`を参照）。
* 期間：1980年Q1〜2021年Q4
* `gdp`：国内総生産（支出側）
* `capital`：資本ストック
* `employed`：就業者
* `hours`：労働者1人あたり月平均就業時間（以下では「平均労働時間」と呼ぶ）
* `total_hours`：総労働時間（`hours`$\times$`employed`）

### 労働時間と雇用の特徴

平均労働時間`hours`，就業者数`employed`，総労働時間`total_hours`の特徴を考えてみる。まずサイクルを計算し，それぞれの変数を図示しよう。（`employed`については対数化した就業者数のトレンドを計算しても良いが，結果は大きく変わらないため，ここでは対数を使わずに計算する。）

In [ ]:
df['hours_trend'] = py4macro.trend(df['hours'])
df['hours_cycle'] = np.log( df['hours']/df['hours_trend'] )

df['employed_trend'] = py4macro.trend(df['employed'])
df['employed_cycle'] = np.log( df['employed']/df['employed_trend'] )

df['total_hours_trend'] = py4macro.trend(df['total_hours'])
df['total_hours_cycle'] = np.log( df['total_hours']/df['total_hours_trend'] )

最初に，平均労働時間を考えよう。

In [ ]:
py4macro.data('jpn-q',description=True)

In [ ]:
df.plot(y=['hours','hours_trend'])
pass

平均労働時間は長期的に減少傾向にある。2020年の平均が100に標準化しているので，1980年代には約25％長かったことを示している。次に，就業者数を図示する。

In [ ]:
df.plot(y=['employed','employed_trend'])
pass

一方，雇用者数は増加傾向にある。女性の労働市場参加率の増加や，雇用形態の変化（非正規など）の影響と考えられる。上の２つの変化を反映したのが総労働時間の変化である。

In [ ]:
df.plot(y=['total_hours','total_hours_trend'])
pass

バブル崩壊後は減少傾向にあるが，過去１０年間に持ち直して来ている。これは雇用の拡大の要因が大きい。

雇用と労働時間の変動（トレンドからの乖離率）を比べてみよう。

In [ ]:
ax_ = df.plot(y=['hours_cycle','employed_cycle'])
ax_.axhline(0, c='red', lw=0.5)
pass

次の２つの点を指摘できる。
* 労働時間の方が変動が大きい。即ち，総労働時間の調整は就業者数よりも労働時間を通して行われていることが伺える。
* 就業者数の変化は、労働時間の変化を後追いする傾向がある。これは就業者の調整費用がより高いためであろう。

自己相関係数を計算してみる。

In [ ]:
cycle_list = ['hours_cycle','employed_cycle','total_hours_cycle']
var_list = ['平均労働時間','就業者数','総労働時間']

print('     自己相関係数\n----------------------')

for c, v in zip(cycle_list,var_list):
    ac = df[c].autocorr()
    print(f'{v:<5}\t{ac:.3f}')  # 1

```{admonition} コードの説明
:class: dropdown

1. `:<5`は`f-string`で代入する変数`v`のスペースを`5`に設定し左寄せ`<`にすることを指定している。また`:.3f`は小数点第三位までの表示を指定している。
```

就業者数がより大きな自己相関係数の値となっており，就業者での調整により時間がかかるためである。即ち，今期の就業者数は前期の就業者数に依存するところが大きいという意味である。

次に，GDPのトレンドからの乖離との相関係数を計算してみる。

In [ ]:
print('GDPサイクルとの相関係数')
print('-------------------------')

for c, v in zip(cycle_list,var_list):
    corr = df[['gdp_cycle',c]].corr().iloc[0,1]
    print(f'{v:<9}\t{corr:.3f}')

やはり非常に強い正の相関があり，平均労働時間の影響が強いようである。

### TFPの特徴

TFPの水準，トレンド，変動（サイクル）を計算するが，資本の所得分配率を次のように仮定する。

In [ ]:
a = 0.36

In [ ]:
# 全要素生産性（対数）の計算
df['tfp_log'] = np.log( df['gdp']/( df['capital']**a * df['total_hours']**(1-a) ) )

# 全要素生産性のトレンド（対数）の計算
df['tfp_log_trend'] = py4macro.trend( df['tfp_log'] )

# 全要素生産性のトレンドからの乖離率の計算
df['tfp_cycle'] = df['tfp_log'] - df['tfp_log_trend']

# 全要素生産性とトレンドのプロット
ax = df.plot(y=['tfp_log','tfp_log_trend'])
ax.set_title('全要素生産性とトレンド（対数）', size=20)
pass

全要素生産性の変動をプロットしよう。

In [ ]:
ax = ( 100 * df['tfp_cycle'] ).plot()
ax.axhline(0, c='red')
ax.set_title('全要素生産性の変動（％）', size=20)
pass

この図から全要素生産性の変動は大きいことが分かる。変動の最小値と最大値を確認してみよう。

In [ ]:
df['tfp_cycle'].min(), df['tfp_cycle'].max()

正の乖離は2％近くあり，負の乖離は4％以上となる。リーマンショック時を除くと正も負も絶対値で概ね2％以内に収まっている。

GDPと一緒にプロットしてみよう。

In [ ]:
ax_ = ( 100 * df[['gdp_cycle','tfp_cycle']] ).plot()
ax_.set_title('GDPとTFPの変動（％）',size=20)
pass

同じ方向に動いているのが確認できる。GDPの変動との相関係数を計算してみると，非常に高いことがわかる。

In [ ]:
df[['gdp_cycle','tfp_cycle']].corr().iloc[0,1]

以前の計算結果を使うと，この数字はGDPのどの構成要素よりも高いことが確認できる。次に，自己相関係数を計算してみよう。

In [ ]:
df['tfp_cycle'].autocorr()

`gdp_cycle`程ではないが、強い持続性があることが確認できる。

In [ ]:
df['gdp_cycle'].autocorr()

＜これらの結果の含意＞

全要素生産性の変動は大きく持続性も高い。またGDPとの相関性も大きい。これらから全要素生産性が景気循環を引き起こす要因ではないかということが示唆される。全要素生産性は資本や労働時間で説明できないGDPの要素であり，そのような「その他」の部分に景気循環を引き起こすランダムな要素が隠されているかも知れない，という考え方である。

### AR(1)としてのTFP

全要素生産性を景気循環の要因として捉えるために，全要素生産性をAR(1)としてモデル化してみようというのがこの節の目的である。そのために`statsmodels`を使い回帰分析をしてみよう。まず１期ずらした変数を作成する。

In [ ]:
df['tfp_cycle_lag'] = df['tfp_cycle'].shift()

```{admonition} コードの説明
:class: dropdown

`shift()`は`Series`を１期ずらすメソッドである。
```

In [ ]:
df[['tfp_cycle','tfp_cycle_lag']].head()

列`tfp_cycle_lag`は列`tfp_cycle`を１期ずらしていることが確認できる。次に回帰分析をおこなう。

In [ ]:
res_tfp = smf.ols('tfp_cycle ~ -1 + tfp_cycle_lag', data=df).fit()
rho = res_tfp.params.iloc[0]
rho

```{admonition} コードの説明
:class: dropdown

回帰式の中に`-1`とあるが、定数項を省く場合に指定する。
```

自己相関係数と殆ど変わらない値である。２つの変数の散布図を表示してみよう。

In [ ]:
df.plot(x='tfp_cycle_lag', y='tfp_cycle', kind='scatter')
pass

正の相関が図でも確認できる。次に$\epsilon_t$について考える。この変数はホワイト・ノイズであり、TFPショックの「源泉」である。平均0の正規分布としてモデル化するが，上の回帰式の残差の標準偏差をTFPの標準偏差の推定値として考えよう。

In [ ]:
sigma = res_tfp.resid.std()
sigma

```{admonition} コードの説明
:class: dropdown

* `res_tfp`の属性`.resid`は残差を返している。
* `.resid`は`Series`であり，その属性`std()`を使い標本標準偏差を計算している（デフォルトでは`ddof=1`)。
```

初期値は0として`rho`と`sigma`を使ってAR(1)の値を計算してプロットしてみよう。

In [ ]:
T = 100        # 標本の大きさ
A = 0          # 初期値
A_list = [A]   # 値を格納するリスト

for i in range(T):
    e = np.random.normal(loc=0,scale=sigma)
    A = rho*A+e
    A_list.append(A)

df_A = pd.Series(A_list)            # Seriesの作成
ax_ = df_A.plot()                   # 図示
ax_.axhline(0, c='red')             # 0の平行線

ac_tfp = df_A.autocorr()       # 自己相関係数の計算
print(f'自己相関係数：{ac_tfp:.3f}')

シミュレーションを行う度に図と自己相関係数は異なることになる。

## 確率的ソロー・モデル

### 説明

上で日本のデータを使い全要素生産性の`rho`と`sigma`を計算しシミュレーションを行なったが，そのような全要素生産性の変動が景気循環の主な要因と考えるのがRBCモデルと呼ばれるものである。RBCモデルの詳細については後の章で検討するが，ここではその導入としてソロー・モデルにAR(1)としてのTFPを組み込んでシミュレーションをおこない，計算結果をデータと比べてどの程度整合性があるかを検討する。

モデルの均衡式として以下を使う。
* 生産関数：$Y_t=A_tB_tK_t^aH^{1-a}$
    * $A_tB_t$が全要素生産性
    * $A_t$は変動を捉える項
    
        $$
        \begin{align*}
        \log(A_{t})&=\rho\log(A_{t-1})+\varepsilon_t \\
        \varepsilon_t&\sim \textit{WH}(0,\sigma^2)
        \end{align*}
        $$

    * $B_t$はトレンドを捉える項
    
        $$
        B_t=B_0(1+g)^t
        $$
    * $H$は一定と仮定する
    
* 資本の蓄積方程式：$K_{t+1}=sY_t+(1-d)K_t$
* 消費：$C_t=(1-s)Y_t$
* 投資（貯蓄）：$I_t=sY_t$

これらの式から次の2式で$K_t$と$A_t$の値が計算できる。

$$
\begin{align*}
K_{t+1}&=sA_tB_tK_t^aH^{1-a}+(1-d)K_t\\
\log(A_{t+1})&=\rho\log(A_{t})+\varepsilon_{t+1}
\end{align*}
$$

変数の初期値を次のように仮定する。
* $A_0=1$
* $K_0=$ 1980年の資本ストックの平均

$H$は次の値として一定とする。
* $H=$ 1980年の総労働時間の平均

パラメータの値を次を仮定する。
* $\rho$と$\sigma$は上で計算した`rho`と`sigma`の値
* $g=$ 1980年から最後の年までの平均四半期成長率
* $s=$ 1980年から最後の年までのGDPに占める投資の割合の平均
* $d=0.025$：年率約10％を想定
* $a=0.36$：資本の所得割合

シミュレーションの結果を次の変数を含む`DataFrame`にまとめる。
* $Y$，$C$，$I$，$K$，$A$の水準の系列
* $Y$，$C$，$I$，$K$，$A$のトレンドの系列
* $Y$，$C$，$I$，$K$，$A$の変動（サイクル）の系列

### 初期値の計算

まずGDP，資本，総労働時間の1980年と2019年の平均を計算する。

In [ ]:
var_list = ['gdp','capital','total_hours']       #1
year_list = [df.index[0].year,df.index[-1].year] #2

gdp_dict = {}  #3
K_dict = {}    #4
H_dict = {}    #5

for yr in year_list:                     #6
    
    cond = ( df.index.year == yr)        #7
    mean = df.loc[cond, var_list].mean() #8
    gdp_dict[yr] = mean.iloc[0]  #9
    K_dict[yr] = mean.iloc[1]    #10
    H_dict[yr] = mean.iloc[2]    #11

```{admonition} コードの説明
:class: dropdown

* `#1`：３つの変数のリスト
* `#2`：平均を計算する年のリストであり，`df.index[0].year`と`df.index[-1].year`はインデックスから最初と最後の年を抽出する
* `#3`：それぞれの年のGDPの平均を格納する辞書
* `#4`：それぞれの年の資本ストックの平均を格納する辞書
* `#5`：それぞれの年の総労働時間の平均を格納する辞書
* `#6`：`year_list`に対しての`for`ループ
* `#7`：`df.index`の年が`yr`と同じ場合は`True`を返し，そうでない場合は`False`を返す`Series`を`cond`に割り当てる。
* `#8`：それぞれの年の行，`var_list`にある列を抽出し平均を計算する。
    * `.mean()`は列の平均を計算する。
    * 計算した平均は`Series`として返されるので，それを左辺にある変数`mean`に割り当てる。
* `#9`：`mean`の0番目の要素を`gdp_dict`のキー`yr`（年）に対応する値をして設定する。
* `#10`：`mean`の1番目の要素を`K_dict`のキー`yr`（年）に対応する値をして設定する。
* `#11`：`mean`の2番目の要素を`H_dict`のキー`yr`（年）に対応する値をして設定する。
```

このコードの結果を確認してみよう。例えば，

In [ ]:
gdp_dict

In [ ]:
from myst_nb import glue
var_list = ['gdp','capital','total_hours']       #1
year_list = [df.index[0].year,df.index[-1].year] #2

gdp_dict = {}  #3
K_dict = {}    #4
H_dict = {}    #5

for yr in year_list:                     #6
    
    cond = ( df.index.year == yr)        #7
    mean = df.loc[cond, var_list].mean() #8
    gdp_dict[yr] = mean.iloc[0]  #9
    K_dict[yr] = mean.iloc[1]    #10
    H_dict[yr] = mean.iloc[2]    #11

start_ = list( gdp_dict.keys() )[0]
end_ = list( gdp_dict.keys() )[1]
glue("start_", start_, display=False)
glue("end_", end_, display=False)

キー・値の２つのペアがある。キー{glue:}`start_`の値には{glue:}`start_`年のGDPの平均が設定されており，同様に，もう一つのキー{glue:}`end_`の値には{glue:}`end_`年のGDPの平均が設定されている。従って，次のコードで{glue:}`start_`年の値にアクセスできる。
```
gdp_dict[1980]
```
他の辞書も同様である。

````{admonition} なぜforループを使うのか。
:class: tip

上のコードで`for`ループを使っているが，もし使わなければ次のようなコードになる。
```
var_list = ['gdp','capital', 'total_hours']
mean1980 = df.loc['1980-01-01':'1980-10-01',var_list].mean()
mean2019 = df.loc['2019-01-01':'2019-10-01',var_list].mean()

gdp1980 = mean1980.iloc[0]
K1980 = mean1980.iloc[1]
H1980 = mean1980.iloc[2]

gdp2019 = mean2019.iloc[0]
K2019 = mean2019.iloc[1]
H2019 = mean2019.iloc[2]
```
同じようなコードが並んでいる。`py4macro`のデータがアップデートされて`2020`年までのデータが含まれることになったとしよう。`for`ループを使うコードであれば`2019`を`2020`に一ヵ所だけ変更すれば良い（変更する必要もないコードを書くことも可能）。`for`ループを使わないコードで何ヵ所修正する必要があるか数えてみよう。面倒と感じないだろうか。それに修正箇所が増えればbugが紛れ込む可能性が増えていく。
````

次に全要素生産性の平均四半期成長率を計算するために`df`の列名を確認しよう。

In [ ]:
df.columns

`gdp`は0番目，`capital`は6番目，`total_hours`は11番目の列にある。この情報をもとに次のように計算しよう。

In [ ]:
var_idx = [0,6,11]         # 1
df0 = df.iloc[0,var_idx]   # 2
df1 = df.iloc[-1,var_idx]  # 3

# 4
B0 = df0['gdp']/( df0['capital']**a * df0['total_hours']**(1-a))

# 5
B1 = df1['gdp']/( df1['capital']**a * df1['total_hours']**(1-a))

no_quarters = len(df)      # 6

g = (B1/B0)**(1/no_quarters)-1  # 7
g

```{admonition} コードの説明
:class: dropdown

1. 3つの変数の列インデックのリスト
2. 最初の行で3つの変数だけを抽出し`df0`に割り当てる。
3. 最後の行で3つの変数だけを抽出し`df1`に割り当てる。
4. 最初の四半期の全要素生産性の計算
5. 最後の四半期の全要素生産性の計算
6. 四半期の数
7. 平均四半期成長率の計算
```

次にGDPに対する投資の割合の平均を計算する。

In [ ]:
s = ( df['investment']/df['gdp'] ).mean()
s

約22％が投資に回されている。

### シミュレーション

#### 関数

シミュレーションのコードを関数としてまとめよう。

In [ ]:
def stochastic_solow(T=160,  # 160=40*4 40年間の四半期の数
                     rho=rho,
                     sigma=sigma,
                     g=g,
                     s=s,
                     d=0.025,
                     a=a,
                     H=H_dict[1980],  # 1980年平均に固定
                     K0=K_dict[1980],
                     B0=B0):
    """引数：
            T:     シミュレーションの回数
            rho:   AR(1)のパラメータ
            sigma: ホワイト・ノイズ分散
            g:     TFPの平均四半期成長率
            s:     貯蓄率
            d:     資本減耗率
            a:     生産関数のパラメータ（資本の所得割合）
            H:     総労働時間
       返り値
            次の変数を含むDataFrame
            GDP，消費，投資，資本ストック，全要素生産性の水準，トレンド，変動"""
    
    # ========== A,B,Kの計算 ==========
    # 計算結果を格納するリスト
    A_list = [1]    # 1
    B_list = [B0]   # 2
    K_list = [K0]   # 3

    # 初期値でありアップデート用変数
    A = 1           # 4
    B = B0          # 5
    K = K0          # 6

    # A,B,Kの時系列の計算
    for t in range(1,T):
        K = s * A*B * K**a *H**(1-a) + (1-d)*K  # 7
        e = np.random.normal(0,scale=sigma)     # 8
        A = A**rho * np.exp(e)                  # 9
        B = B0*(1+g)**t                         # 10
        
        A_list.append(A)                        # 11
        B_list.append(B)
        K_list.append(K)
    
    # ========== DataFrameの作成 ==========
    df_sim = pd.DataFrame({'K':K_list,          # 12
                           'A':A_list,
                           'B':B_list})

    # ========== Y,C,Iの計算 ==========          # 13
    df_sim['Y'] = df_sim['A']*df_sim['B']*df_sim['K']**a * H**(1-a)
    df_sim['C'] = (1-s)*df_sim['Y']
    df_sim['I'] = s*df_sim['Y']
    
    # ========== トレンドとサイクルの計算 ==========
    for v in ['K','A','Y','C','I']:
        # ---------- レンドの計算 ----------      # 14
        df_sim[f'{v}_log_trend'] = py4macro.trend( np.log(df_sim[v]) )
        # ---------- サイクルの計算 ----------    # 15
        df_sim[f'{v}の変動'] = np.log( df_sim[v] ) - df_sim[f'{v}_log_trend']
    
    return df_sim

```{admonition} コードの説明
:class: dropdown

1. `A`の値を格納する空のリスト。初期値が入っている。
2. `B`の値を格納する空のリスト。初期値が入っている。
3. `K`の値を格納する空のリスト。初期値が入っている。
4. `for`ループでアップデートされる`A`の値を一次的に格納する。最初は初期値が入っている。
5. `for`ループでアップデートされる`B`の値を一次的に格納する。最初は初期値が入っている。
6. `for`ループでアップデートされる`K`の値を一次的に格納する。最初は初期値が入っている。
7. 右辺では(4)~(6)の値（`A`，`B`，`K`）を使い来期の`K`を計算し，その値を左辺の`K`に割り当てる。この瞬間に(6)の`K`がアップデートされる。
8. 来期のホワイト・ノイズの値を生成し`e`に割り当てる。
9. 右辺では(4)の値（`A`）を使い来期の`A`の値を計算し，その値を左辺の`A`に割り当てる。この瞬間に(4)の`A`がアップデートされる。
10. 右辺では来期の`B`の値を計算し，その値を左辺の`B`に割り当てる。この瞬間に(5)の`B`がアップデートされる。
11. リスト(1)~(3)に来期値を追加する。
12. リスト(1)~(3)を使いDataFrameの作成
13. `Y`，`C`，`I`を計算し新たな列としてDataFrameに追加
14. トレンドを計算し新たな列としてDataFrameに追加
    * `f-string`を使って新しく作る列名を`K_trend`のように`_trend`を追加する。
15. トレンドからの乖離を計算し新たな列としてDataFrameに追加
    * `f-string`を使って新しく作る列名を`K_cycle`のように`_cycle`を追加する。
```

#### 実行

デフォルトの値でシミュレーションを行い、最初の5行を表示する。

In [ ]:
df_sim = stochastic_solow()
df_sim.head()

$Y$の水準とトレンドの図示（対数）。

In [ ]:
np.log( df_sim['Y'] ).plot(title='シミュレーション：Yとトレンド（対数）')
df_sim['Y_log_trend'].plot()
pass

トレンドの傾きは成長率を表しているが，徐々に減少していることが分かる。最初は成長率に対する資本蓄積の効果が大きいが，定常状態に近づくにつれて資本蓄積の効果が減少しているためである。

$Y$の変動（サイクル）の図示。

In [ ]:
ax_ = ( 100 * df_sim['Yの変動'] ).plot(title='シミュレーション：Yの変動（％）')
ax_.axhline(0, c='red')
pass

絶対値で約2％内に収まった変動となっている。データでは，リーマンショック時はマイナス約4％程乖離したが，それ以外は絶対値で2％以内に収まっており，シミュレーションは概ねデータと同じような特徴を持っていると言える。

#### 結果

それぞれの変数の自己相関係数を計算してみる。

In [ ]:
var_list = ['Y','C','I','K','A']

print('\n--- シミュレーション：変動の自己相関係数 ---\n')

for v in var_list:
    
    ac = df_sim[f'{v}の変動'].autocorr()
    print(f'{v}の変動： {ac:.3f}')

まずシミュレーションを行う度にこれらの値は変化することに留意し結果を考えよう。ある程度の自己相関が発生しており，$A$の影響が反映されている。$Y$，$C$，$I$の値は同じなのは，$C$，$I$は$Y$の線形関数であるためであり，これらの３つの変数は$A$の変動と似た動きになっている。また$K$の値が大きいのは、ストック変数であるため変化に時間が掛かるためある。データ`df`に含まれる変数の自己相関係数と比べるために，まず消費，投資，資本の変動を計算する。

In [ ]:
df.columns

In [ ]:
data_var_list = ['consumption','investment','capital']

for v in data_var_list:
    
    df[f'{v}_cycle'] = np.log( df[v]/py4macro.trend(df[v]) )

結果を示すために，`data_var_list`の最初と最後に`gdp`と`tfp`を追加する。

In [ ]:
data_var_list = ['gdp']+data_var_list+['tfp']

In [ ]:
print('\n--- データ：変動の自己相関係数 ---\n')

for v in data_var_list:
    
    ac = df[f'{v}_cycle'].autocorr()
    print(f'{v:>11}の変動： {ac:>5.3f}')      # 1

```{admonition} コードの説明
:class: dropdown

1. `f-string`を使って変数`v`と`ac`を代入している。
    * `:>11`は`v`を表示する幅を`11`に設定し右寄せ（`>`）にしている。
    * `:>5`は`ac`を表示する幅を`5`に設定して右寄せ（`>`）にしている。
    * `.3f`は`ac`を小数点第三位までを表示するように指定している。
```

実際の資本ストックは非常に大きな持続性があり、シミュレーションの結果はそれを捉えている。しかし他の変数に関しては持続性が足りないようである。

しかしシミュレーションの結果はランダム変数の実現値に依存しているので，シミュレーションを行う度に異なる結果が出てくる。また，このシミュレーションでは`T=160`として40年間を考えたが，`T`の値によっても結果は大きく左右される。`T`を大きくするとより安定的な結果になっていくだろう。例えば，`T=10_000`を試してみよう。

次に、$Y$との相関係数を計算する。

In [ ]:
print('\n--- シミュレーション：GDPとの相関係数 ---\n')

for v in var_list[1:]:
    cov = df_sim[['Yの変動',f'{v}の変動']].corr().iloc[0,1]
    print(f'{v}の変動： {cov:.3f}')

* $C$と$I$の値は1.0となっている理由は$Y$と線形になっているためであり，全く同じように変動するためである。
* $Y$と$A$は非常に強く相関している。生産関数を見ると，両変数も線形の関係があり，$K$の変化によって変化のズレが生じるメカニズムとなっているためである。

相関度をデータと比べてみよう。

In [ ]:
print('\n--- データ：GDPとの相関係数 ---\n')

for v in data_var_list[1:]:
    cor = df[['gdp_cycle',f'{v}_cycle']].corr().iloc[0,1]
    print(f'{v:>11}の変動：{cor:>5.3f}')

$C$と$I$に関してはシミュレーションの相関係数は大きすぎる結果となっている。

確率的ソロー・モデルによるシミュレーション結果は，マクロ変数の特徴を捉えている部分もあるが景気循環モデルとして考えるには難しいだろう。消費者の効用最大化問題が欠落しているため，$Y$，$C$，$I$が全く同じように動くのがネックになり，消費や投資の特徴を捉えることができていない。後の章で議論するが，消費者の最適化問題や将来の予測などを導入したRBCモデルは，よりデータに近い結果となる。しかし，いくつか問題が残るのも事実である。
* TFPショックとは何なのか？
    * 生産関数を見る限りTFPは生産性を表している。技術水準，政府の規制，政治システムや経済制度の変化を捉えると考えられ，例えば，ビジネスに対する規制の強化によって負のショックが発生する。しかし，それだけで実際に観測されるGDPの大きな変動（例えば、リーマン・ショックやコロナ・ショック）を説明するのには疑問である。
* ソロー・モデルも含めて価格調整が瞬時に行われるが，次の問題が残る。
    * 古典派の二分法が成立すると仮定するが、短期的には無理がある仮定（例えば、賃金）。
    * 景気循環に対して金融政策は無力なのか？
    
このような問題意識を持って，RBCモデルとニューケインジアン・モデルを議論していくことにする。